In [ ]:
from vllm import LLM, SamplingParams

In [ ]:
LLM_MODEL = "Qwen/Qwen2.5-3B-Instruct"
NUM_GPUs = 1

In [ ]:
llm = LLM(model=LLM_MODEL, tensor_parallel_size=NUM_GPUs, dtype="half")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

WARNING 12-30 10:31:20 config.py:2276] Casting torch.bfloat16 to torch.float16.
INFO 12-30 10:31:33 config.py:510] This model supports multiple tasks: {'generate', 'score', 'classify', 'embed', 'reward'}. Defaulting to 'generate'.
INFO 12-30 10:31:33 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='Qwen/Qwen2.5-3B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=Fa

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 12-30 10:33:53 model_runner.py:1099] Loading model weights took 5.7915 GB
INFO 12-30 10:34:02 worker.py:241] Memory profiling takes 9.33 seconds
INFO 12-30 10:34:02 worker.py:241] the current vLLM instance can use total_gpu_memory (14.75GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 12-30 10:34:02 worker.py:241] model weights take 5.79GiB; non_torch_memory takes 0.13GiB; PyTorch activation peak memory takes 2.52GiB; the rest of the memory reserved for KV Cache is 4.83GiB.
INFO 12-30 10:34:02 gpu_executor.py:76] # GPU blocks: 8791, # CPU blocks: 7281
INFO 12-30 10:34:02 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 4.29x
INFO 12-30 10:34:06 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizati

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:30<00:00,  1.14it/s]

INFO 12-30 10:34:37 model_runner.py:1535] Graph capturing finished in 31 secs, took 0.95 GiB
INFO 12-30 10:34:37 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 44.66 seconds


In [ ]:
sampling_params = SamplingParams(top_k=50, max_tokens=500, top_p=0.9, stop="<end>")

In [ ]:
def create_json_prompt_for_synthetic_data(**kwargs):
    attributes = {key: value for key, value in kwargs.items() if value != "n/a"}

    prompt = """
**Objective:**
Produce **one** realistic text passage that includes clearly identified named entities. Each entity should be meticulously labeled according to its type for straightforward extraction.

**Format Requirements:**
- The output should be formatted in JSON, containing the text and the corresponding entities list.
- Each entity in the text should be accurately marked and annotated in the 'entities' list.
- Meticulously follow all the listed attributes.
- **Important:** Return **only one JSON object**, without any additional text or explanations.

**Entity Annotation Details:**
- All entity types must be in lowercase. For example, use "type" not "TYPE".
- Entity types can be multiwords separate by space. For instance, use "entity type" rather than "entity_type".
- Entities spans can be nested within other entities.
- A single entity may be associated with multiple types. list them in the key "types".

**Output Schema:**

<start attribute_1="value1" attribute_2="value2" ...>
{
  "text": "{text content}",
  "entities": [
    {"entity": "entity name", "types": "type_name"},
    ...
  ]
}
<end>
"""
    return prompt

In [ ]:
import re
def clean_json_output(raw_output):
    match = re.search(r'\{.*\}', raw_output, re.DOTALL)
    if match:
        return match.group(0)
    return None

In [ ]:
import json

def generate(**kwargs):
    prompt = create_json_prompt_for_synthetic_data(**kwargs)
    outputs = llm.generate([prompt], sampling_params)
    raw_output = outputs[0].outputs[0].text
    print("Raw output:", raw_output)
    cleaned_output = clean_json_output(raw_output)
    if cleaned_output:
        try:
            return json.loads(cleaned_output)
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)
            return None
    else:
        print("Failed to extract JSON from output")
        return None

In [ ]:
def create_dataset(num_samples, output_file):
    dataset = []

    for i in range(num_samples):
        print(f"Generating sample {i + 1}/{num_samples}")
        result = generate(language="english", types_of_text="detailed job ads", sector="machine learning", country="USA")
        if result:
            dataset.append(result)

    with open(output_file, 'w') as f:
        for item in dataset:
            f.write(json.dumps(item) + "\n")

    print(f"Dataset saved to {output_file}")

In [ ]:
NUM_SAMPLES = 100
OUTPUT_FILE = "synthetic_dataset_vllm.jsonl"

create_dataset(NUM_SAMPLES, OUTPUT_FILE)

Generating sample 1/100




Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts:  50%|█████     | 1/2 [00:03<00:03,  3.34s/it, est. speed input: 72.95 toks/s, output: 58.30 toks/s]

Processed prompts: 100%|██████████| 2/2 [00:04<00:00,  2.08s/it, est. speed input: 117.51 toks/s, output: 78.26 toks/s]


Raw output: {
  "text": "tim berners-lee, the father of the world wide web, was born in london in 1955 and currently works as a researcher at cern. berners-lee's invention has revolutionized the way we communicate and access information, making the world more connected.",
  "entities": [
    {"entity": "tim berners-lee", "types": "person"},
    {"entity": "the father of the world wide web", "types": "organization"},
    {"entity": "london", "types": "location"},
    {"entity": "1955", "types": "date"},
    {"entity": "cern", "types": "organization"},
    {"entity": "berners-lee", "types": "person"},
    {"entity": "world wide web", "types": "product"},
    {"entity": "world", "types": "location"}
  ]
}
Generating sample 2/100




Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.09s/it, est. speed input: 34.46 toks/s, output: 33.76 toks/s]


Raw output: {
  "text": "barack obama, who was born on october 4, 1961 in kunstler, quran, hawaii, u.s., is a prominent political leader and has held the role of president of the united states of america since 2009. he is known for his strong advocacy on environmental issues and his policies that promote diversity and inclusion.",
  "entities": [
    {"entity": "barack obama", "types": "person"},
    {"entity": "october 4, 1961", "types": "date"},
    {"entity": "kunstler, quran, hawaii, u.s.", "types": "location"},
    {"entity": "president of the united states of america", "types": "position"},
    {"entity": "2009", "types": "date"},
    {"entity": "barack obama", "types": "person"},
    {"entity": "environmental issues", "types": "topic"},
    {"entity": "diversity and inclusion", "types": "policy"}
  ]
}
Generating sample 3/100


...

Raw output: {
  "text": "on tuesday june 19th the weather in paris is expected to be 25 degrees celsius with a high of 28 degrees celsius and a low of 22 degrees celsius, these conditions are good for sightseeing, especially in the eiffel tower area, which experienced a significant increase in tourist numbers on the same day, as a result of the city's summer festival celebrations. this event marks the beginning of a two-month celebration that includes fireworks, concerts, and various exhibitions, all designed to enhance the city's tourism industry. for those interested in the event, the local tourism office has detailed information on the activities and schedules available.",
  "entities": [
    {"entity": "paris", "types": "city"},
    {"entity": "eiffel tower", "types": "landmark"},
    {"entity": "june 19th", "types": "date"},
    {"entity": "summer festival", "types": "event"},
    {"entity": "local tourism office", "types": "organization"}
  ]
}
Generating sample 99/100




Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.87s/it, est. speed input: 41.64 toks/s, output: 33.28 toks/s]


Raw output: {
  "text": "mark zuckerberg, the founder of facebook, was recently quoted as saying that he believes artificial intelligence will be the future of the internet and that it will play a crucial role in solving humanity's greatest challenges. he further elaborated by mentioning that he is particularly interested in areas such as deep learning and natural language processing, which he believes will enable more sophisticated interactions with computers and enhance the user experience. additionally, he talked about his desire to see ai be more accessible to a broader audience and to use it to benefit society at large.",
  "entities": [
    {"entity": "mark zuckerberg", "types": "person"},
    {"entity": "facebook", "types": "organization"},
    {"entity": "artificial intelligence", "types": "product"},
    {"entity": "deep learning", "types": "product"},
    {"entity": "natural language processing", "types": "product"}
  ]
}
Generating sample 100/100




Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.71s/it, est. speed input: 36.38 toks/s, output: 33.54 toks/s]

Raw output: ```json
{
  "text": "alice wants to buy a new bike in the city park tomorrow, john will join him at 2 PM sharp, and sue will come a bit later in the afternoon",
  "entities": [
    {"entity": "alice", "types": "person"},
    {"entity": "john", "types": "person"},
    {"entity": "sue", "types": "person"},
    {"entity": "city park", "types": "location"}
  ]
}
``` The JSON formatted response as per the requirements is as follows:

```json
{
  "text": "alice wants to buy a new bike in the city park tomorrow, john will join him at 2 PM sharp, and sue will come a bit later in the afternoon",
  "entities": [
    {"entity": "alice", "types": "person"},
    {"entity": "john", "types": "person"},
    {"entity": "sue", "types": "person"},
    {"entity": "city park", "types": "location"}
  ]
}
```
JSON decode error: Extra data: line 10 column 1 (char 353)
Dataset saved to synthetic_dataset_vllm.jsonl
